# Advent of Code 2023, Day 16
[Day 16 Challenge](https://adventofcode.com/2023/day/16)

In [605]:
import aoc
import numpy as np

%reload_ext autoreload

day = 16
sample = True
sample_number = 1

In [606]:
input_list = aoc.split_contents(aoc.read_input(f'Input/day_{day:02}{"_sample"+str(sample_number) if sample else ""}.txt'))

# Part 1

In [607]:
def pretty_print(array):
    # Prints the array with the bottom left corner being 0,0
    array_t = array.T
    print('*' * array_t.shape[1])
    for y in reversed(range(array_t.shape[1])):
        print(''.join(array_t[y]))

In [608]:
# Generate the contraption as a 2d NumPy array, bottom left is 0,0
# The y-axis increases up
# The x-axis increases right
row_list = list()
for i,input_row in enumerate(input_list):
    row_list.append([x for x in input_row])
contraption = np.array(row_list[::-1]).T

In [609]:
pretty_print(contraption)

**********
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....


Beam direction is in degrees. Up is 0, right is 90, down is 180, and left is 270.

In [610]:
beams = dict()
beams[0] = {'start': (0,contraption.shape[1]-1), 'current_direction': 90, 'path':[(0,contraption.shape[1]-1)]}

In [611]:
def get_object(vector, direction, coord):
    """
    For a given NumPy vector, from the coord tuple, return the first object found looking in the given direction.
    If no object is found in the laser path, return x,y coordinates outside of the contraption with the '#' object.
    :param vector: 1d NumPy vector
    :param direction: 0,90,180,270 up, right, down, left
    :param coord: Tuple with x,y coordinates
    :return: x, y coordinates of the object and the object
    """
    vector_list = list(vector)
    index = -1
    for index in range(len(vector_list)):
        if vector_list[0] == '.':
            vector_list.pop(0)
        else:
            obj = vector_list[0]
            break
    
    if len(vector_list) == 0:
        # Beam edge condition
        obj = '#'
        index += 1

    match direction:
        case 0:
            x,y = coord[0], coord[1]+index+1
        case 90:
            x,y = coord[0]+index+1, coord[1]
        case 180:
            x,y = coord[0], coord[1]-index-1
        case 270:
            x,y = coord[0]-index-1, coord[1]
    return x, y, obj


def turn(direction, left_or_right):
    direction += 90 if left_or_right == 'R' else -90
    direction = direction % 360
    return direction

def follow_path(beam):
    match beam['current_direction']:
        case 0:
            beam_vector = contraption[beam['start'][0], beam['start'][1]+1::]
        case 90:
            beam_vector = contraption[beam['start'][0]+1::, beam['start'][1]]
        case 180:
            beam_vector = contraption[beam['start'][0], 0:beam['start'][1]][::-1]
        case 270:
            beam_vector = contraption[0:beam['start'][0], beam['start'][1]][::-1]
        case _:
            beam_vector = None
    x, y, obj = get_object(beam_vector, beam['current_direction'], beam['start'])
    
    # Fill in the beam path with #s
    if x != beam['start'][0]:
        path = [(x_path, beam['start'][1]) for x_path in range(min(x,beam['start'][0]), max(x,beam['start'][0])+1)]
    else:
        path = [(beam['start'][0], y_path) for y_path in range(min(y,beam['start'][1]), max(y,beam['start'][1])+1)]
    
    # aoc.logger.info(f'{path=}')
    
    new_beams = list()
    match obj:
        case '\\':
            match beam['current_direction']:
                case 90 | 270:
                    left_or_right='R'
                case _:
                    left_or_right = 'L'
            new_beams.append({'current_direction': turn(beam['current_direction'], left_or_right), 'start': (x,y), 'edge': False, 'path': path})
        case '/':
            match beam['current_direction']:
                case 90 | 270:
                    left_or_right='L'
                case _:
                    left_or_right = 'R'
            new_beams.append({'current_direction': turn(beam['current_direction'], left_or_right), 'start': (x,y), 'edge': False, 'path': path})
        case '|':
            match beam['current_direction']:
                case 90 | 270:
                    # if beam enters from left or right, beam splits up and down
                    new_beams.append({'current_direction': 0, 'start': (x,y), 'edge': False, 'path': path})
                    new_beams.append({'current_direction': 180, 'start': (x,y), 'edge': False, 'path': path})
                case _:
                    # if beam enters from top or bottom, beam passes through
                    new_beams.append({'current_direction': beam['current_direction'], 'start': (x,y), 'edge': False, 'path': path})
        case '-':
            match beam['current_direction']:
                case 0 | 180:
                    # if beam enters from top or bottom, beam splits left and right
                    new_beams.append({'current_direction': 90, 'start': (x,y), 'edge': False, 'path': path})
                    new_beams.append({'current_direction': 270, 'start': (x,y), 'edge': False, 'path': path})
                case _:
                    # if beam enters from left or right, beam passes through
                    new_beams.append({'current_direction': beam['current_direction'], 'start': (x,y), 'edge': False, 'path': path})             
        case '#':
            new_beams.append({'current_direction': beam['current_direction'], 'start': (x,y), 'edge': True, 'path': path})
    return new_beams

In [612]:
def test_turns():
    # Test the turn function
    assert turn(0, 'L') == 270
    assert turn(90,'L') == 0
    assert turn(180,'L') == 90
    assert turn(270,'L') == 180
    assert turn(0, 'R') == 90
    assert turn(90,'R') == 180
    assert turn(180,'R') == 270
    assert turn(270,'R') == 0
    
def test_follow_path():
    # Test the beam splitters
    # Test the up/down split coming at a | from the left
    assert follow_path({'start': (0,9), 'current_direction': 90}) == [{'start': (1,9), 'current_direction': 0, 'edge': False,
                                                                       'path':[(0,9),(1,9)]},
                                                                      {'start': (1,9), 'current_direction': 180, 'edge': False,
                                                                       'path':[(0,9),(1,9)]}]
    # Test the up/down split coming at a | from the right
    assert follow_path({'start': (2,9), 'current_direction': 270}) == [{'start': (1,9), 'current_direction': 0, 'edge': False,
                                                                       'path':[(1,9),(2,9)]},
                                                                      {'start': (1,9), 'current_direction': 180, 'edge': False,
                                                                       'path':[(1,9),(2,9)]}]
    # Test the left/right split coming at a - from the top
    assert follow_path({'start': (2,9), 'current_direction': 180}) == [{'start': (2,8), 'current_direction': 90, 'edge': False,
                                                                       'path':[(2,8),(2,9)]},
                                                                       {'start': (2,8), 'current_direction': 270, 'edge': False,
                                                                       'path':[(2,8),(2,9)]}]
    # Test the left/right split coming at a - from the bottom
    assert follow_path({'start': (2,7), 'current_direction': 0}) == [{'start': (2,8), 'current_direction': 90, 'edge': False,
                                                                       'path':[(2,7),(2,8)]},
                                                                     {'start': (2,8), 'current_direction': 270, 'edge': False,
                                                                       'path':[(2,7),(2,8)]}]
    
    # Test the beam splitter pass through
    # Test a pass through coming at a | from the bottom
    assert follow_path({'start': (1,8), 'current_direction': 0}) == [{'start': (1,9), 'current_direction': 0, 'edge': False,
                                                                      'path':[(1,8),(1,9)]}]
    # Test a pass through coming at a | from the top
    assert follow_path({'start': (5,8), 'current_direction': 180}) == [{'start': (5,7), 'current_direction': 180, 'edge': False,
                                                                        'path':[(5,7),(5,8)]}]
    # Test a pass through coming at a - from the left
    assert follow_path({'start': (1,8), 'current_direction': 90}) == [{'start': (2,8), 'current_direction': 90, 'edge': False,
                                                                       'path':[(1,8),(2,8)]}]
    # Test a pass through coming at a - from the right
    assert follow_path({'start': (3,8), 'current_direction': 270}) == [{'start': (2,8), 'current_direction': 270, 'edge': False,
                                                                        'path':[(2,8),(3,8)]}]
    
        
    # Test the mirrors
    # Test a right turn coming at a \ from the left
    assert follow_path({'start': (3,8), 'current_direction': 90}) == [{'start': (4,8), 'current_direction': 180, 'edge': False,
                                                                       'path':[(3,8),(4,8)]}]
    # Test a right turn coming at a \ from the right
    assert follow_path({'start': (5,8), 'current_direction': 270}) == [{'start': (4,8), 'current_direction': 0, 'edge': False,
                                                                        'path':[(4,8),(5,8)]}]
    # Test a left turn coming at a \ from the bottom
    assert follow_path({'start': (4,7), 'current_direction': 0}) == [{'start': (4,8), 'current_direction': 270, 'edge': False,
                                                                      'path':[(4,7),(4,8)]}]
    # Test a left turn coming at a \ from the top
    assert follow_path({'start': (4,9), 'current_direction': 180}) == [{'start': (4,8), 'current_direction': 90, 'edge': False,
                                                                        'path':[(4,8),(4,9)]}]
    
    # Test a left turn coming at a / from the left
    assert follow_path({'start': (3,3), 'current_direction': 90}) == [{'start': (4,3), 'current_direction': 0, 'edge': False,
                                                                       'path':[(3,3),(4,3)]}]
    # Test a left turn coming at a / from the right
    assert follow_path({'start': (5,3), 'current_direction': 270}) == [{'start': (4,3), 'current_direction': 180, 'edge': False,
                                                                       'path':[(4,3),(5,3)]}]
    # Test a right turn coming at a / from the top
    assert follow_path({'start': (4,4), 'current_direction': 180}) == [{'start': (4,3), 'current_direction': 270, 'edge': False,
                                                                        'path':[(4,3),(4,4)]}]
    # Test a right turn coming at a / from the bottom
    assert follow_path({'start': (4,2), 'current_direction': 0}) == [{'start': (4,3), 'current_direction': 90, 'edge': False,
                                                                      'path':[(4,2),(4,3)]}]
    
    # Test the edges
    # Test going off the right edge
    assert follow_path({'start': (9,5), 'current_direction': 90}) == [{'current_direction': 90, 'start': (10, 5), 'edge': True,
                                                                       'path': [(9,5),(10,5)]}]
    # Test going off the left edge
    assert follow_path({'start': (0,5), 'current_direction': 270}) == [{'current_direction': 270, 'start': (-1, 5), 'edge': True,
                                                                        'path': [(-1,5),(0,5)]}]
    # Test going off the top edge
    assert follow_path({'start': (0,9), 'current_direction': 0}) == [{'current_direction': 0, 'start': (0, 10), 'edge': True,
                                                                      'path': [(0,9),(0,10)]}]
    # Test going off the bottom edge
    assert follow_path({'start': (0,0), 'current_direction': 180}) == [{'current_direction': 180, 'start': (0, -1), 'edge': True,
                                                                        'path': [(0,-1),(0,0)]}]


In [613]:
test_follow_path()

In [614]:
follow_path({'start': (0,9), 'current_direction': 90})

[{'current_direction': 0,
  'start': (1, 9),
  'edge': False,
  'path': [(0, 9), (1, 9)]},
 {'current_direction': 180,
  'start': (1, 9),
  'edge': False,
  'path': [(0, 9), (1, 9)]}]

In [622]:
path_symbols = {0:'^', 90:'>', 180:'v', 270:'<'}
beam_counter = 0
contraption_path = np.copy(contraption)
v = {'start': (8,4), 'current_direction': 180}
while True:
    old_v = v
    v = follow_path(v)[0]
    aoc.logger.info(f'{v["path"]=}')
    for coord in v['path']:
        try:
            # Need to handle the case when the x or y coordinate is negative. This is off the left or bottom edge.
            if coord[0] >= 0 and coord[1] >=0:
                contraption_path[coord[0]][coord[1]] = path_symbols[old_v['current_direction']]
        except IndexError:
            aoc.logger.info(f'Edge of contraction reached')
    aoc.logger.info(f'{beam_counter=} {old_v=} -> {v=}')
    pretty_print(contraption_path)
    if v['edge']:
        break
    beam_counter += 1

INFO:AoC:v["path"]=[(8, -1), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4)]
INFO:AoC:beam_counter=0 old_v={'start': (8, 4), 'current_direction': 180} -> v={'current_direction': 180, 'start': (8, -1), 'edge': True, 'path': [(8, -1), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4)]}


**********
.|...\....
|.-.\.....
.....|-...
........|.
..........
........v\
..../.\\v.
.-.-/..|v.
.|....-|v\
..//.|..v.


# Part 2